# A dialogue with GPT-3 using the OpenAI API
TODO:
- DEVELOP
    - save conversation within the get_chatting() as dictionary `{'initial': '', 'human': ['','', ... ''], 'AI': ['','', ... '']}`
    - build display function using this dictionary setup
    - export conversation when it ends
- DEPLOY
    - binder
    - voila
    - heroku
- SHARE

In [ ]:
import os
import openai
from getpass import getpass

from IPython.display import clear_output, display, HTML
import ipywidgets as widgets

In [ ]:
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
def pretty_print(x):
    return display( HTML( x.replace("\n","<br>") ) )

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# set OpenAI key
openai.api_key = os.getenv('OPENAI_API_KEY')
if openai.api_key is None:
    openai.api_key = getpass('Create an OpenAI API key at https://beta.openai.com/account/api-keys and paste here:\n')

In [ ]:
def get_API_response(prompt):
    
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        temperature=0.9,
        max_tokens=150,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0.6,
        stop=[" Human:", " AI:"]
    )
    ai_response = response.choices[0].text
    
    return ai_response

In [ ]:
def get_chatting(initial = ''):
    
    # create the text box widget
    text_box = widgets.Text(
        placeholder='Enter your prompt here',
        layout = widgets.Layout(width='1000px', height='auto') #set width and height
    )

    # create the reset button widget
    reset_button = widgets.Button(description='Reset thread')

    # define a function that will be called when the user submits a prompt
    def on_submit(text):
            
        # add human prompt + prepare for API
        prompt = initial + '\n\nHuman: ' + text_box.value + '\n\nAI:'
        prompt += ' ' + get_API_response(prompt)
        
        # clear output and keep going with updated prompt
        clear_output()
        get_chatting(prompt)
        
    # tell the text box to add labels
    text_box.on_submit(on_submit)

    # when reset button is clicked clear output, extract initial prompt and restart
    def on_reset_clicked(b):
        clear_output()
        initial_prompt = initial.split('\n\nHuman:')[0] # extract beginning of the conversation
        get_chatting(initial_prompt)
        
    # call the on_reset_clicked function when the reset_button is clicked
    reset_button.on_click(on_reset_clicked)
            
    # create a horizontal box layout to arrange the text_box and reset_button next to each other
    hbox = widgets.HBox([text_box, reset_button])
    hbox.layout.width = '1000px'
    
    # initial text
    pretty_print(initial)
    display(hbox)
    text_box.focus()

In [ ]:
initial_prompt = """The following is a counseling session between an AI mental health professional and a human. The AI is warm, accepting and empathic. It is a strong advocate of journaling and is of the opinion that journaling is an effective tool for advancing mental health. The AI is likely to recommend intelligent prompts to help people get started with journaling.

AI: Hello, how may I help you today?"""

get_chatting(initial_prompt)